In [1]:
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import KNeighborsClassifier
import sklearn
from __future__ import print_function
import librosa
import numpy as np
import matplotlib.pyplot as plt
from librosa import display,core
from dtw import dtw
from numpy import linalg as LA
from math import sqrt
from scipy import stats
from sklearn import svm
from sklearn.svm import SVR
from mlxtend.plotting import plot_decision_regions
import matplotlib.patches as mpatches
from sklearn.model_selection import cross_val_predict, cross_val_score,cross_validate
from sklearn import metrics
from mlxtend.plotting import plot_confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

from collections import Counter

from imblearn.over_sampling import RandomOverSampler
#%matplotlib inline

Audio Waveform (a):

The term "audio waveform" refers to the representation of an audio signal in the time domain. It is a continuous representation of the audio signal, showing how the sound pressure (amplitude) varies over time.
In digital audio processing, the audio waveform is typically discretized or sampled at regular intervals to create a digital representation of the continuous analog audio signal.

Sample Rate (s):

The "sample rate," denoted as s in the code, refers to the number of samples (audio measurements) taken per second during the analog-to-digital conversion of an audio signal. It is expressed in Hertz (Hz) and is a fundamental parameter in digital audio processing.
The sample rate determines the granularity at which the continuous audio waveform is captured digitally. A higher sample rate provides more detail but requires more data storage and processing power.

In [2]:
import librosa
import numpy as np
from sklearn.model_selection import train_test_split
import os

def createTrainingData(start, end):
    samples = []
    telangana_dir = 'audio/Telangana'
    coastal_dir = 'audio/Coastal'
    rayalseema_dir = 'audio/Rayalseema'
    
    regions = {
        'Telangana': telangana_dir,
        'Coastal': coastal_dir,
        'Rayalseema':rayalseema_dir
    }

    for region, region_dir in regions.items():
        if region == 'Telangana':
            file_range = range(239, 357)
        elif region == 'Rayalseema':
            file_range = range(116, 215)
        elif region == 'Coastal':
            file_range = range(1, 102)  # Assuming audio files in Coastal are named from '1.wav' to '101.wav'

        for i in file_range:
            ind = 0
            try:
                file_path = os.path.join(region_dir, f"{i}.wav")
                a, s = librosa.load(file_path)
            except:
                ind = 1

            if ind == 0:
                mfcc = librosa.feature.mfcc(y=a, sr=s)
                temp = mfcc.T[1][start:end]
                for frame in range(10, 50):
                    temp = np.concatenate((temp, mfcc.T[frame][start:end]))
                samples.append([temp, region])
    
    print(len(samples))
    X = [i[0] for i in samples]
    Y = [j[1] for j in samples]
    return X, Y



In [3]:
clf4 = svm.SVC()
data,label = createTrainingData(1,19)
X_train, X_test, y_train, y_test = train_test_split(data, label, test_size = 0.2)

C:\Users\Lavanya Tetakali\AppData\Local\Temp\ipykernel_23692\3362901313.py:30: UserWarning: PySoundFile failed. Trying audioread instead.
  a, s = librosa.load(file_path)
C:\Users\Lavanya Tetakali\AppData\Local\Programs\Python\Python310\lib\site-packages\librosa\core\audio.py:183: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


317


In [4]:
from sklearn.svm import SVC

def predict_svm(X, Y, Z):
    svm = SVC()
    svm.fit(X, Y)
    prediction = svm.predict(Z)
    scores = cross_val_score(svm, data, label)  # You should have 'data' and 'label' defined somewhere in your code.
    print(scores.mean())
    return prediction

def printPrediction(prediction,actual_label):
    print("1 -> Telangana, 0-> Coastal, 2-> Rayalseema\n")
    for i in range(len(prediction)):
        print("Actual ->",actual_label[i]," Prediction ->",prediction[i])

def accuracy(prediction,actual):
    count=0
    l=len(prediction)
    for i in range(l):
        if(prediction[i]==actual[i]):
            count+=1
    return count/l*100

In [5]:
Counter(label)

Counter({'Telangana': 118, 'Coastal': 100, 'Rayalseema': 99})

In [6]:
X_train, X_test, y_train, y_test  = train_test_split(data,label)
ros = RandomOverSampler(random_state=0)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)
print(sorted(Counter(y_resampled).items()))

[('Coastal', 91), ('Rayalseema', 91), ('Telangana', 91)]


In [7]:
clf = svm.SVC()
clf.fit(X_resampled,y_resampled)
y_pred=clf.predict(X_test)
print("Accuracy:",accuracy(y_pred,y_test))

Accuracy: 97.5


In [8]:
conf_matrix = confusion_matrix(y_test, y_pred)
conf_matrix

array([[23,  0,  1],
       [ 0, 29,  0],
       [ 0,  1, 26]], dtype=int64)

In [9]:
# Access the elements of the confusion matrix
tn = conf_matrix[0, 0]  # True negatives
fp = conf_matrix[0, 1]  # False positives
fn = conf_matrix[1, 0]  # False negatives
tp = conf_matrix[1, 1]  # True positives

In [10]:
precision = round(tp/(tp+fp),4)
recall = round(tp/(tp+fn),4)
fmeasure = round(2*((precision*recall)/(precision+recall)),4)
rej = round(tn/(tn+fp),4)
acc = round((tn+tp)/(tn+tp+fp+fn),4)

In [11]:
print(precision,recall,fmeasure,rej,acc)

1.0 1.0 1.0 1.0 1.0


In [13]:
import joblib  # Used to save and load models


# Save the trained SVM model to a PKL file
pkl_file = "svm_accent_model.pkl"  # Filename for your model
joblib.dump(clf, pkl_file)  # Save the model

print(f"Model saved to {pkl_file}")


Model saved to svm_accent_model.pkl


1. **Precision**: Measures how many of the positive predictions made by the model were actually correct. A precision of 1.0 means that when the model predicts a sample as belonging to a class, it is almost always correct.

2. **Recall (Sensitivity)**: Measures how many of the actual positive samples were correctly identified by the model. A recall of 1.0 indicates that the model is very good at identifying positive samples.

3. **F1-Score**: The harmonic mean of precision and recall. It provides a balance between precision and recall. An F1-score of 1.0 suggests a perfect balance between precision and recall.

4. **Rejection Rate (rej)**: The proportion of true negatives out of the total negatives (true negatives + false positives). A rejection rate of 1.0 means that all negatives are correctly identified as negatives.

5. **Overall Accuracy (acc)**: The proportion of correct classifications out of all predictions. An accuracy of 1.0 indicates that all predictions were correct.

In [18]:
# Required Python Libraries
import os
import librosa
import numpy as np
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
import joblib
import warnings

# Ignore specific warnings
warnings.filterwarnings("ignore", message="PySoundFile failed. Trying audioread instead.")
warnings.filterwarnings("ignore", category=FutureWarning)

# Feature Extraction Function
def extract_features(audio_path, start=1, end=19):
    a, s = librosa.load(audio_path, sr=None)
    mfcc = librosa.feature.mfcc(y=a, sr=s)
    feature_vector = mfcc.T[1][start:end]  # Ensure consistent range of MFCCs
    for frame in range(10, 50):
        feature_vector = np.concatenate((feature_vector, mfcc.T[frame][start:end]))
    return feature_vector

# Create Training Data
def create_training_data(start=1, end=19):
    samples = []
    regions = {
        'Telangana': 'audio/Telangana',
        'Coastal': 'audio/Coastal',
        'Rayalseema': 'audio/Rayalseema'
    }
    
    for region, region_dir in regions.items():
        if region == 'Telangana':
            file_range = range(239, 357)
        elif region == 'Rayalseema':
            file_range = range(116, 215)
        elif region == 'Coastal':
            file_range = range(1, 102)
        
        for i in file_range:
            try:
                file_path = os.path.join(region_dir, f"{i}.wav")
                feature_vector = extract_features(file_path, start, end)
                samples.append([feature_vector, region])
            except Exception as e:
                print(f"Error processing {file_path}: {str(e)}")
    
    # Split data into features and labels
    X = [i[0] for i in samples]
    Y = [j[1] for j in samples]
    return X, Y

# Create and Train the SVM Classifier
data, label = create_training_data()
X_train, X_test, y_train, y_test = train_test_split(data, label, test_size=0.2)

# Oversample the training set to avoid class imbalance
ros = RandomOverSampler(random_state=0)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)

# Train the SVM classifier
clf = SVC()
clf.fit(X_resampled, y_resampled)

# Save the trained model to a PKL file
pkl_file = "svm_accent_model.pkl"
joblib.dump(clf, pkl_file)

# Predict and Evaluate
y_pred = clf.predict(X_test)

# Compute Accuracy
def accuracy(prediction, actual):
    return np.mean(np.array(prediction) == np.array(actual)) * 100

# Print the accuracy
print("Accuracy:", accuracy(y_pred, y_test), "%")

# Function to Predict Accent with SVM
def predict_accent(model, audio_file):
    feature_vector = extract_features(audio_file)
    prediction = model.predict([feature_vector])
    return prediction

# Test the Prediction with the Saved Model
audio_file = "audio/Rayalseema/117.wav"  # Example audio file
accent = predict_accent(clf, audio_file)
print("Predicted Accent:", accent)


Error processing audio/Coastal\100.wav: [Errno 2] No such file or directory: 'audio/Coastal\\100.wav'
Accuracy: 87.5 %
Predicted Accent: ['Rayalseema']


In [14]:
from sklearn.model_selection import cross_val_score

# Your existing code for data loading and oversampling

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data, label, test_size=0.2)

# Oversample the training set using RandomOverSampler
ros = RandomOverSampler(random_state=0)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)

# Create and train the SVM classifier
clf = svm.SVC()
data,label = createTrainingData(1,19)
clf.fit(X_resampled, y_resampled)

# Perform cross-validation
cv_scores = cross_val_score(clf, data, label, cv=10) # You can adjust the number of folds with the 'cv' parameter

# Print the mean cross-validation score
print("Mean Cross-Validation Score:", np.mean(cv_scores))

# Make predictions on the test set
y_pred = clf.predict(X_test)

# Calculate and print accuracy
print("Accuracy:", accuracy(y_pred, y_test))

# Calculate and print other performance metrics (precision, recall, fmeasure, rejection, overall accuracy)
conf_matrix = confusion_matrix(y_test, y_pred)
tn = conf_matrix[0, 0]
fp = conf_matrix[0, 1]
fn = conf_matrix[1, 0]
tp = conf_matrix[1, 1]

precision = round(tp / (tp + fp), 4)
recall = round(tp / (tp + fn), 4)
fmeasure = round(2 * ((precision * recall) / (precision + recall)), 4)
rej = round(tn / (tn + fp), 4)
acc = round((tn + tp) / (tn + tp + fp + fn), 4)

print("Precision:", precision)
print("Recall:", recall)
print("F-Measure:", fmeasure)
print("Rejection:", rej)
print("Overall Accuracy:", acc)


C:\Users\Lavanya Tetakali\AppData\Local\Temp\ipykernel_23692\3362901313.py:30: UserWarning: PySoundFile failed. Trying audioread instead.
  a, s = librosa.load(file_path)
C:\Users\Lavanya Tetakali\AppData\Local\Programs\Python\Python310\lib\site-packages\librosa\core\audio.py:183: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


317
Mean Cross-Validation Score: 0.9267137096774194
Accuracy: 95.3125
Precision: 1.0
Recall: 1.0
F-Measure: 1.0
Rejection: 1.0
Overall Accuracy: 1.0


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import librosa.display

# Example usage:
file_path_sample = 'audio/Telangana/240.wav'
audio_features = extract_audio_features(file_path_sample)
y, sr = librosa.load(file_path_sample, sr=None)  # Load audio file

# Plotting pitch-related features
plt.figure(figsize=(10, 16))

# Plot speech signal waveform
plt.subplot(5, 1, 1)
librosa.display.waveshow(y, sr=sr)
plt.title('Speech Signal Waveform')
plt.xlabel('Time (s)')
plt.ylabel('Amplitude')

# Plot pitch contour
plt.subplot(5, 1, 2)
pitches, magnitudes = librosa.core.piptrack(y=y, sr=sr)
pitches = [librosa.hz_to_midi(p) for p in pitches[pitches > 0]]
times = librosa.times_like(np.array(pitches))  # Convert pitches to NumPy array
plt.plot(times, pitches, label='Pitch (MIDI)')
plt.title('Pitch Contour')
plt.xlabel('Time (s)')
plt.ylabel('Pitch (MIDI)')

# Plot energy
plt.subplot(5, 1, 3)
energy = np.cumsum(np.square(y))
plt.plot(librosa.times_like(energy), energy)
plt.title('Energy')
plt.xlabel('Time (s)')
plt.ylabel('Energy')

# Plot power spectral density (PSD)
plt.subplot(5, 1, 4)
psd = librosa.feature.spectral_centroid(y=y, sr=sr)[0]
plt.plot(librosa.times_like(psd), psd)
plt.title('Power Spectral Density (PSD)')
plt.xlabel('Time (s)')
plt.ylabel('PSD')

# Plot intensity
plt.subplot(5, 1, 5)
intensity = np.cumsum(np.abs(y))
plt.plot(librosa.times_like(intensity), intensity)
plt.title('Intensity')
plt.xlabel('Time (s)')
plt.ylabel('Intensity')

plt.tight_layout()
plt.show()


In [ ]:
import librosa
import numpy as np

def extract_audio_features(file_path, sr=22050):
    # Load audio file
    y, sr = librosa.load(file_path, sr=sr)

    # Extract pitch-related features using librosa
    pitches, magnitudes = librosa.core.piptrack(y=y, sr=sr)
    pitches = np.array([librosa.hz_to_midi(p) for p in pitches[pitches > 0]])

    min_pitch = np.min(pitches)
    max_pitch = np.max(pitches)
    pitch_range = max_pitch - min_pitch
    avg_pitch = np.mean(pitches)
    std_pitch = np.std(pitches)
    mean_abs_slope_pitch = np.mean(np.abs(np.diff(pitches)))

    # Extract other features
    energy = np.sum(y**2)
    psd = librosa.feature.spectral_centroid(y=y, sr=sr)
    intensity = np.sum(np.abs(y))
    
    # Extract formants (F1, F2, F3)
    formants = librosa.effects.harmonic(y)
    f1 = formants[0]
    f2 = formants[1]
    f3 = formants[2] if len(formants) > 2 else None

    return {
        'Min Pitch (Hz)': librosa.midi_to_hz(min_pitch),
        'Max Pitch (Hz)': librosa.midi_to_hz(max_pitch),
        'Pitch Range (Hz)': pitch_range,
        'Average Pitch (Hz)': librosa.midi_to_hz(avg_pitch),
        'Std Dev Pitch (Hz)': librosa.midi_to_hz(std_pitch),
        'Mean Abs Slope Pitch (Hz)': librosa.midi_to_hz(mean_abs_slope_pitch),
        'Energy': energy,
        'Power Spectral Density (PSD)': np.mean(psd),
        'Intensity': intensity,
        'F1 (Hz)': librosa.midi_to_hz(f1),
        'F2 (Hz)': librosa.midi_to_hz(f2),
        'F3 (Hz)': librosa.midi_to_hz(f3) if f3 is not None else None
    }

# Example usage:
file_path = 'audio/Telangana/245.wav'
audio_features = extract_audio_features(file_path)
print(audio_features)


In [ ]:
import librosa
import numpy as np

def extract_audio_features(file_path, sr=22050):
    # Load audio file
    y, sr = librosa.load(file_path, sr=sr)

    # Extract pitch-related features using librosa
    pitches, magnitudes = librosa.core.piptrack(y=y, sr=sr)
    pitches = np.array([librosa.hz_to_midi(p) for p in pitches[pitches > 0]])

    min_pitch = np.min(pitches)
    max_pitch = np.max(pitches)
    pitch_range = max_pitch - min_pitch
    avg_pitch = np.mean(pitches)
    std_pitch = np.std(pitches)
    mean_abs_slope_pitch = np.mean(np.abs(np.diff(pitches)))

    # Extract other features
    energy = np.sum(y**2)
    psd = librosa.feature.spectral_centroid(y=y, sr=sr)
    intensity = np.sum(np.abs(y))
    
    # Extract formants (F1, F2, F3)
    formants = librosa.effects.harmonic(y)
    f1 = formants[0]
    f2 = formants[1]
    f3 = formants[2] if len(formants) > 2 else None

    return {
        'Min Pitch (Hz)': librosa.midi_to_hz(min_pitch),
        'Max Pitch (Hz)': librosa.midi_to_hz(max_pitch),
        'Pitch Range (Hz)': pitch_range,
        'Average Pitch (Hz)': librosa.midi_to_hz(avg_pitch),
        'Std Dev Pitch (Hz)': librosa.midi_to_hz(std_pitch),
        'Mean Abs Slope Pitch (Hz)': librosa.midi_to_hz(mean_abs_slope_pitch),
        'Energy': energy,
        'Power Spectral Density (PSD)': np.mean(psd),
        'Intensity': intensity,
        'F1 (Hz)': librosa.midi_to_hz(f1),
        'F2 (Hz)': librosa.midi_to_hz(f2),
        'F3 (Hz)': librosa.midi_to_hz(f3) if f3 is not None else None
    }

# Example usage:
file_path = 'audio/Coastal/15.wav'
audio_features = extract_audio_features(file_path)
print(audio_features)


In [ ]:
import librosa
import numpy as np

def extract_audio_features(file_path, sr=22050):
    # Load audio file
    y, sr = librosa.load(file_path, sr=sr)

    # Extract pitch-related features using librosa
    pitches, magnitudes = librosa.core.piptrack(y=y, sr=sr)
    pitches = np.array([librosa.hz_to_midi(p) for p in pitches[pitches > 0]])

    min_pitch = np.min(pitches)
    max_pitch = np.max(pitches)
    pitch_range = max_pitch - min_pitch
    avg_pitch = np.mean(pitches)
    std_pitch = np.std(pitches)
    mean_abs_slope_pitch = np.mean(np.abs(np.diff(pitches)))

    # Extract other features
    energy = np.sum(y**2)
    psd = librosa.feature.spectral_centroid(y=y, sr=sr)
    intensity = np.sum(np.abs(y))
    
    # Extract formants (F1, F2, F3)
    formants = librosa.effects.harmonic(y)
    f1 = formants[0]
    f2 = formants[1]
    f3 = formants[2] if len(formants) > 2 else None

    return {
        'Min Pitch (Hz)': librosa.midi_to_hz(min_pitch),
        'Max Pitch (Hz)': librosa.midi_to_hz(max_pitch),
        'Pitch Range (Hz)': pitch_range,
        'Average Pitch (Hz)': librosa.midi_to_hz(avg_pitch),
        'Std Dev Pitch (Hz)': librosa.midi_to_hz(std_pitch),
        'Mean Abs Slope Pitch (Hz)': librosa.midi_to_hz(mean_abs_slope_pitch),
        'Energy': energy,
        'Power Spectral Density (PSD)': np.mean(psd),
        'Intensity': intensity,
        'F1 (Hz)': librosa.midi_to_hz(f1),
        'F2 (Hz)': librosa.midi_to_hz(f2),
        'F3 (Hz)': librosa.midi_to_hz(f3) if f3 is not None else None
    }

# Example usage:
file_path = 'audio/Rayalseema/125.wav'
audio_features = extract_audio_features(file_path)
print(audio_features)
